In [1]:
# Import libaries
import pandas as pd
from bs4 import BeautifulSoup
import requests

### Step 1: Create a soup object from the home page

In [2]:
url = 'http://chubbygrub.com/'
res = requests.get(url)
res.status_code

200

In [3]:
soup = BeautifulSoup(res.content, 'lxml')

### Step 2: Scrape the home page soup for every restaurant

Note: Your best bet is to create a list of dictionaries, one for each restaurant. Each dictionary contains the restaurant's name and slug. The result of your scrape should look something like this:

```python
restaurants = [
    {'name': 'A&W Restaurants', 'slug': 'aw-restaurants'}, 
    {'name': "Applebee's", 'slug': 'applebees'},
    ...
]
```

In [4]:
a = soup.find_all('a', {'class' : 'btn btn-lg btn-primary'})
names = [ai.text for ai in a]
# names

In [5]:
slugs = [ai.attrs['href'].replace('/restaurants/','') for ai in a]

In [6]:
restaurants = []
for k, v in zip(names, slugs):
    dictionary = {}
    dictionary['name'] = k
    dictionary['slugs'] = v
    restaurants.append(dictionary)
    
# restaurants

### Step 3: Using the slug, scrape each restaurant's page and create a single list of food dictionaries.

Your list of foods should look something like this:
```python
foods = [
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    ...
]
```

**Note**: Remove extra white space from each category

In [7]:
base_url = 'http://chubbygrub.com'

In [8]:
urls_list = [base_url + ai.attrs['href'] for ai in a]

In [9]:
# testing out for one page: 
url = 'http://chubbygrub.com/restaurants/kfc/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'lxml')

In [10]:
table = soup.find('table', {'id' : 'items'})

In [11]:
food = []
for row in table.find_all('tr')[1:]:
#     print(row)
    items = {}
    
    items['calories'] = row.find('td', {'itemprop' : 'calories'}).text
    items['carbs'] = row.find('td', {'itemprop' : 'carbohydrateContent'}).text
    items['category'] = row.find('td', {'class' : 'hidden-xs category-column'}).text.strip()
    items['fat'] = row.find('td', {'itemprop' : 'fatContent'}).text
    items['name'] = row.find('td', {'itemprop' : 'name'}).text
    items['restaurant'] = soup.find('span', {'itemprop' : 'name'}).text
    
    food.append(items)
# food 

In [17]:
from time import sleep #or import time, then later time.sleep(seconds)

In [18]:
food = []
for u in urls_list:
    res = requests.get(u)
    sleep(1)
    soup = BeautifulSoup(res.content, 'lxml')
    table = soup.find('table', {'id' : 'items'})
    
    for row in table.find_all('tr')[1:]:
        items = {}    
        items['calories'] = row.find('td', {'itemprop' : 'calories'}).text
        items['carbs'] = row.find('td', {'itemprop' : 'carbohydrateContent'}).text
        items['category'] = row.find('td', {'class' : 'hidden-xs category-column'}).text.strip()
        items['fat'] = row.find('td', {'itemprop' : 'fatContent'}).text
        items['name'] = row.find('td', {'itemprop' : 'name'}).text
        items['restaurant'] = soup.find('span', {'itemprop' : 'name'}).text
        
        food.append(items)

### Step 4: Create a pandas DataFrame from your list of foods

**Note**: Your DataFrame should have 4,977 rows

In [19]:
df = pd.DataFrame(food)

In [20]:
df.shape

(4977, 6)

In [21]:
df.head(2)

,calories,carbs,category,fat,name,restaurant
0,0,0,Drinks,0,A&W® Diet Root Beer,A&W Restaurants
1,0,0,Drinks,0,A&W® Diet Root Beer,A&W Restaurants


### Step 5: Export to csv

**Note:** Don't export the index column from your DataFrame

In [22]:
%pwd

'/Users/hayatoumy/Documents/portfolio/webscraping-1'

In [23]:
df.to_csv('food_items.csv', index = False) #if index is true, adds another id column!